# Pima Indians Onset of Diabetes Prediction

The objective of the dataset is to diagnostically predict whether or not a patient has diabetes, based on the following 8 diagnostic measurements.

    Pregnancies: Number of times pregnant
    GlucosePlasma: glucose concentration a 2 hours in an oral glucose tolerance test
    BloodPressure: Diastolic blood pressure (mm Hg)
    SkinThicknessTriceps: skin fold thickness (mm)
    Insulin: 2-Hour serum insulin (mu U/ml)
    BMI: Body mass index (weight in kg/(height in m)^2)
    DiabetesPedigreeFunction: Diabetes pedigree function
    Age: Age (years)

The dataset includes data from 768 women beyond the age of 21. The target variable:

    Outcome: Class variable (0 or 1), where 1 corresponds to having diabetes. 

### Goals of this notebook

* Look deeper into various ensembling methods and compare their performance.

**Classifiers used:**
    * Dummy Classifiers
    * KNN
    * Decision Tree 
    * Bagging Classifier
    * Random Forests
    * Sklearn's GBM
    * XgBoostClassifier 
    * Voting classifier
    * Stacking Classifier
    
 
 ### Notebook Structure
     * Import Libraries and Load data
     * Perform Exploratory Data Analysis and Data Cleaning
         -Here we will also look into Principal Component Analysis
     * Build ensemble Models 
     * Compare the different ensemble models

## Import Libraries and Load  data

In [ ]:
# Load the dependencies

import numpy as np
import pandas as pd 
import os

#Import sklearn classes 
from sklearn.model_selection import train_test_split,RepeatedKFold, cross_val_score,KFold, RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier,export_graphviz
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
# sklearn utility to compare algorithms
from sklearn import model_selection

#Visualisation Libraries
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz 

import warnings  
warnings.filterwarnings('ignore')

from eli5 import explain_weights,show_weights
from yellowbrick import ROCAUC
from yellowbrick.classifier import ClassificationReport

print("Imported all libraries successfully")
print(os.listdir("../input"))

CV_N_REPEATS=20
BINS=10

In [ ]:
df = pd.read_csv("../input/diabetes.csv")
df.head()

In [ ]:
print('Shape of the dataset', df.shape)

## EDA and Data Cleaning

In [ ]:
# We can use the pandas_profiling library to automate most of the EDA process for us. It has been commented out for keeping the notebook concise. 
#import pandas_profiling
#report=pandas_profiling.ProfileReport(df,check_correlation =True);
#report.to_file(outputfile="eda_report.html")

We first check if the data is shuffled. We can do that by plotting the Outcome against the dataframe index (in x-axis).

In [ ]:
plt.figure()
plt.plot(df.Outcome,'.')
plt.xlabel('Index')
plt.ylabel('Outcome')

The above plot confirms that the dataframe rows are shuffled. That is good. We next check for class imbalance. As can be seen from the below figure, there's fewer samples with diabeties than the ones not having diabetes. To tackle this imbalance, we would make use of Stratified sampling during the model building phase.

In [ ]:
plt.figure()
ax = sns.countplot(data=df, x='Outcome');
ax.set_title("Seaborn countplot of Dibaetes rates within the dataset");

We check the distribution of the 8 attributes against the target variable Outcome.  

In [ ]:
f, axes = plt.subplots(2, 4,figsize=(15,15))
sns.set(style="white", palette="Set3", color_codes=True)
sns.boxplot(  y="Pregnancies", x= "Outcome", data=df,  orient='v', ax=axes[0,0])
sns.boxplot(  y="Glucose", x= "Outcome", data=df,  orient='v' , ax=axes[0,1])
sns.boxplot(  y="BloodPressure", x= "Outcome", data=df,  orient='v' ,ax=axes[0,2])
sns.boxplot(  y="SkinThickness", x= "Outcome", data=df,  orient='v' , ax=axes[0,3])
sns.boxplot(  y="Insulin", x= "Outcome", data=df,  orient='v' ,  ax=axes[1,0])
sns.boxplot(  y="BMI", x= "Outcome", data=df,  orient='v' ,  ax=axes[1,1])
sns.boxplot(  y="DiabetesPedigreeFunction", x= "Outcome", data=df,  orient='v' , ax=axes[1,2])
sns.boxplot(  y="Age", x= "Outcome", data=df,  orient='v' ,  ax=axes[1,3])

f.subplots_adjust(left=0.08, right=0.98, bottom=0.05, top=0.9,
                    hspace=0.4, wspace=0.3)
#f.suptitle('Distribution of data')
plt.tight_layout()

From the above plot, we notice that no particular attribute clearly differentiates the two classes, though with attribute Glucose, the two classes seem to have a noticeable difference in their samples distribution. This variable could serve as a good differentiatior. 

We also note that the dataset contains samples with improbable values, such as samples which have zeros for the following attributes: BMI, Glucose, SkinThickness,Insulin,and BloodPressure. That does not make sense. Clearly, there might have been an error while recording the data or some data values might not have been recorded at all and instead filled with zeros. 

Next, we clean the data using basic techniques. We are not using any specialised method for data cleaning primarily because that is not the focus of this notebook. 

#### Data Cleaning
First we replace zeros with nan since after that counting them would be easier and zeros need to be replaced with suitable values.

In [ ]:
df_copy = df.copy(deep = True)
df_copy[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = df_copy[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.NaN)
print('Number of zero entries in each attribute:\n')
print(df_copy.isnull().sum())

To fill these Nan values the data distribution needs to be understood.

In [ ]:
p = df.hist(figsize = (15,15))

In [ ]:
#Impute NaN values using mean and meadian values
print('Imputing NaN values')
df_copy['Glucose'].fillna(df_copy['Glucose'].mean(), inplace = True)
df_copy['BloodPressure'].fillna(df_copy['BloodPressure'].mean(), inplace = True)
df_copy['BMI'].fillna(df_copy['BMI'].mean(), inplace = True)
df_copy['SkinThickness'].fillna(df_copy['SkinThickness'].median(), inplace = True)
df_copy['Insulin'].fillna(df_copy['Insulin'].median(), inplace = True)
print('Number of zero entries in each attribute:\n')
print(df_copy.isnull().sum())

This confirms we have no zero values in our data anymore. We continue exploring the distribution of the attributes and the relation between our attributes. Below is a correlation plot from which we gauge into the linear relationship between our attributes.

In [ ]:
#plot the correlation map of the dataset  
plt.figure(figsize=(10,10))
corr = df_copy.corr()
corr.index = df_copy.columns
sns.heatmap(corr, annot = True, cmap='RdYlGn', vmin=-1, vmax=1)
plt.title("Correlation Heatmap", fontsize=16)
plt.show()

Pregnancies and Age are positively correlated, so are BMI and SkinThickness. As for the target variable, Glucose seems to be most linearly correlated with it. To further understand the bivariate relationships of the data, we draw pairplots.

In [ ]:
plt.figure()
sns.pairplot(data=df_copy,hue='Outcome',diag_kind='kde', palette='deep');

Along the diagonal we have distributions of the attributes for the two classes (Diabetic/Non-Diabetic). These plots confirm that no attribute clearly separates the two distributions. 

In this dataset, we have a total of 768 samples spread in 8 dimensions (8 attributes). That's a huge dimension space for such few samples. So we'll next try to reduce the dimensionality of our data into their principal components and see if the two classes can be segregated. We use PCA for dimensionality reduction.

### Principal Component Analysis (PCA)
PCA is used to decompose a multivariate dataset in a set of successive orthogonal components that explain a maximum amount of the variance.

In [ ]:
# Separating the features and the target (Y) 
X = df_copy.iloc[:,0:8]
Y = df_copy.iloc[:,8]

#Standardizing the features
X= StandardScaler().fit_transform(X)
# Fit PCA and transform X.
pca=PCA(n_components=.90)
pca.fit(X)
print('Variance explained by the principal components(in decreasing order): ',pca.explained_variance_ratio_)
#print('PCA singular values: ',pca.singular_values_)
X1=pca.transform(X)
print('Shape of transformed X: ',X1.shape)

Notice that we need 7 principal components to explain 90% of the variance in the data. Plotting the first two principal components below, which together explain approximately 47% of the total variance, we observe that the two classes have been differentiated to some extent but not enough.  

In [ ]:
plt.figure()
sns.scatterplot(X1[:,0],X1[:,1], hue=Y, palette='deep')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Scatter Plot of First two Principal Component')
plt.show()

How about plotting the first 3 principal components in a 3D plot and check how well our data is classified. These 3 components together explain 61% of the variance in the data.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
plt.style.use('classic')
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X1[:,0],X1[:,1], X1[:,2], c=df.Outcome, s=30)
ax.view_init(10, 100)
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
plt.tight_layout()
plt.show()

As we can see, the two classes have not been well segregated even after projecting the data into new dimensions using PCA. So I infer that using the PCA-transformed data for further model building won't help much in improving classification performance. We will continue to use the original data.

## Model Building
In this section, we build several machine learning models and later analyse their results.
#### DummyClassifier
We start with the most basic, a dummy classifier which predicts the most frequent class at all times. This would serve as our baseline. 
Split X,y into train and test sets; We use the original data instead of the PCA transformed data.

In [ ]:
X = df_copy.iloc[:,0:8]
Y = df_copy.iloc[:,8]
seed = 7
test_size = 0.20
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)

In [ ]:
dum=DummyClassifier(strategy='most_frequent')
dum=dum.fit(X_train,y_train)

#compute accuracy
score=dum.score(X_test, y_test)
print("Dummy Classifier Accuracy: %.2f%%" % (score * 100.0))

In [ ]:
strategy = "most_frequent"

scores = cross_val_score(dum,X, Y, 
                         cv=RepeatedKFold(n_repeats=CV_N_REPEATS), 
                         scoring=None) 
scores_dummy = scores.copy()

score_line = "Scores (Accuracy) mean={0:.2f} +/- {1:.2f} (1 s.d.)".format(scores.mean(),scores.std())
plt.figure(figsize=(7,7))
fig, ax = plt.subplots()
pd.Series(scores).hist(ax=ax, bins=BINS)
ax.set_title(f"RepeatedKFold ({len(scores)} folds) with DummyClassifier({strategy})\n" + score_line);
ax.set_xlabel("Score")
ax.set_ylabel("Frequency");

In [ ]:
# Helper functions for graphical plotting of decision trees and to plot confusion matrix 
def plot_tree_graph(model,columns,class_names):
    #This function plots the constructed decision tree
    dot_data = export_graphviz(model,feature_names=columns,class_names=class_names) 
    graph = graphviz.Source(dot_data) 
    return graph


def confusion_mat(y_pred,y_test):
    plt.figure()
    sns.set(font_scale=1.5)
    cm = confusion_matrix(y_pred, y_test)
    sns.heatmap(cm, annot=True, fmt='g')
    plt.title('Confusion matrix', y=1.1)
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')
    plt.show()

Check the variance of our DummyClassifier's predictions

#### K-Nearest Neighbors

In [ ]:
knn=KNeighborsClassifier(n_neighbors=11) 
knn.fit(X_train,y_train)

#compute accuracy
scores = cross_val_score(knn, X, Y, cv=RepeatedStratifiedKFold(n_repeats=CV_N_REPEATS))
print(f"Accuracy mean={scores.mean():0.2f} +/- {scores.std():0.2f} (1 s.d.)")

#### Decision tree 

We next train a decision tree classifier and check the accuracy results. 

In [ ]:
dt=DecisionTreeClassifier(random_state=1, max_depth=2)
dt=dt.fit(X_train,y_train)
dt_scores = cross_val_score(dt, X, Y, cv=RepeatedStratifiedKFold(n_repeats=CV_N_REPEATS))
print(f"Accuracy mean={dt_scores.mean():0.2f} +/- {dt_scores.std():0.2f} (1 s.d.)")

### Plot decision tree to visualize the splittling rules

In [ ]:
plt.figure()
graph=plot_tree_graph(dt,X.columns,class_names=['0','1'])
graph

### Bagging Classifier
This classifier fits base classifiers each on random subsets of the original dataset and then aggregate their individual predictions (either by voting or by averaging) to form a final prediction. 
Such a meta-estimator can typically be used as a way to reduce the variance of a black-box estimator (e.g., a decision tree), by introducing randomization into its construction procedure and then making an ensemble out of it.

In [ ]:
bag=BaggingClassifier(n_estimators=100,oob_score=True) 
bag=bag.fit(X_train,y_train)

bag_scores = cross_val_score(bag, X, Y, cv=RepeatedStratifiedKFold(n_repeats=CV_N_REPEATS))
print("Accuracy mean={0:0.2f} +/- {1:0.2f} (1 s.d.)".format(scores.mean(),scores.std()))
print("Out of bag score: {0:0.2f}".format(bag.oob_score_*100) );

### Random Forest

In [ ]:
num_estimators=100
rf = RandomForestClassifier(n_estimators=num_estimators) 
rf.fit(X_train, y_train)

rf_score=rf.score(X_test, y_test)
print("Accuracy of Random Forest Classifier: {0:0.2f}".format(rf_score * 100.0));

#Make Predictions
y_pred = rf.predict(X_test)
#Plot the confusion matrix
confusion_mat(y_pred, y_test)

From the random forest model, we have 35 incorrectly labeled samples. In medical data analaysis, as it is usually the case, we are more concerned about the False Negatives (or Misses), i.e. diabetic samples who have been incorrectly labelled as non-diabetic. This model results in 15 such cases. 

We use the eli5 library to analyse which are the most important features for our learned RF model

In [ ]:
feature_names=X_train.columns.values
show_weights(rf,feature_names=feature_names)

As expected, Glucose turns out to be the most important feature for our RF model. This affirms our intuition formed during the data exploration phase. 
#### Check variance in RF prediction quality


In [ ]:
scores = cross_val_score(rf, X, Y, cv=RepeatedStratifiedKFold(n_repeats=CV_N_REPEATS))
scores_est = scores.copy()
print(f"Scores mean={scores.mean():0.2f} +/- {scores.std():0.2f} (1 s.d.)")

score_line = f"Scores (Accuracy) mean={scores.mean():0.2f} +/- {scores.std():0.2f} (1 s.d.)"
plt.figure()
fig, ax = plt.subplots()
pd.Series(scores).hist(ax=ax, bins=BINS)
ax.set_title(f"RepeatedKFold ({len(scores)} folds) with RandomForest\n" + score_line);
ax.set_xlabel("Score")
ax.set_ylabel("Frequency");

 We compare the prediction performance of our random forest classifier against the Dummy classifier. 

In [ ]:
plt.figure()
fig, ax = plt.subplots()
df_dummy_est_scores = pd.DataFrame({'dummy': scores_dummy, 'RF': scores_est})
df_dummy_est_scores.plot(kind='hist', ax=ax, bins=20)
ax.set_xlabel("Score")
ax.set_title("Dummy vs RandomForest Scores");

Starting with a dummy classifier, we can confirm that we have made some progress in the right direction to predict for onset of diabetes. 

### GradientBoostingClassifier (Sklearn)
This is an ensemble model based on the boosting paradigm, i.e. sequential model building using several weak classifiers. We start with 500 estimators and a decision tree classifier of depth 4 as our weak learner. 

In [ ]:
from sklearn.metrics import mean_squared_error
params={'n_estimators': 500,'learning_rate': 0.01,'max_depth': 4, 'loss':'deviance'}
gbm=GradientBoostingClassifier(**params)
gbm.fit(X_train, y_train)

Let's see how the model deviance performs w.r.t the number of estimators. Deviance is the logistic loss function used in all implementation of GBM and is the default loss function for classification problems. 

In [ ]:
# compute test set deviance
test_score = np.zeros((params['n_estimators'],), dtype=np.float64)

for i, y_pred in enumerate(gbm.staged_predict(X_test)):
    test_score[i] = gbm.loss_(y_test, y_pred)

#plot train and test set deviance against the number of estimators
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title('GBM Deviance w.r.t Number of Estimators')
plt.plot(np.arange(params['n_estimators']) + 1, gbm.train_score_, 'b-',label='Training Set Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, test_score, 'r-',
         label='Test Set Deviance')
plt.legend(loc='best')
plt.xlabel('Boosting Iterations')
plt.ylabel('Deviance')

As we can see from the plot above that the test set deviance plateaus at around 1.3 at about 100 estimators. Having more estimators would not help in predictions, in fact it would lead model to overfit the training data.
We set the number of estimators to 100 and retrain our GBM. We also plot the confusion matrix to check for errors. As can be seen, GBM does a good job at reducing the False Positives to 7 from 12 in the random forest model, though the number of false detections has increased to 33., False detections are samples that are not diabetic but have been classified as diabetic.

In [ ]:
params={'n_estimators': 100,'learning_rate': 0.01,'max_depth': 4, 'loss':'deviance'}
gbm=GradientBoostingClassifier(**params)
gbm.fit(X_train, y_train)

# make predictions for test data
y_pred = gbm.predict(X_test)

# evaluate predictions
gbm_score = accuracy_score(y_test, y_pred)
print("Accuracy of GBM Classifier: {0:0.2f}".format(gbm_score * 100.0));

#Plot the confusion matrix
confusion_mat(y_pred, y_test)

Plot Feature Importance for the GBM model

In [ ]:
feature_importance = gbm.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
plt.subplot(1, 2, 2)
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos, df.columns[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()

#### XgBoost

This model is an optimized variant of the Gradient boosting models, which at its core does the same work as the previous Gradient Boosting machine does. The difference is that XgBoost algorithm is developed with both deep consideration in terms of systems optimization and principles in machine learning. The goal of the library is to push the extreme of the computation limits of machines to provide a scalable, portable and accurate library. 

In [ ]:
from xgboost import XGBClassifier, plot_importance,to_graphviz

# fit model on training data
param = {'max_depth': 3, 'eta': 0.8, 'subsample':1, 'objective': 'binary:logistic'}
xgb = XGBClassifier(**param)
xgb.fit(X_train, y_train)

In [ ]:
# make predictions for test data
y_pred = xgb.predict(X_test)

# evaluate predictions
xgb_score = accuracy_score(y_test, y_pred)
print("Accuracy of XGB Classifier: {0:0.2f}".format(xgb_score * 100.0));

#Plot the confusion matrix
confusion_mat(y_pred, y_test)

XGBoost in most cases performs better than GBM.  As we can see here the classification accuracy has increased to 83.12% than compared to GBM's 74%, but the Misses have increased from 7 in GBM to 12 with Xgboost model.  

Next we plot Feature Importance based on the Xgboost model. From the feature importance graphs for all models plotted until now, we see Glucose is the most important feature. Importance order for the other features is more or less same.

In [ ]:
# plot feature importance using built-in function
plt.figure()
plot_importance(xgb,title="Feature importance from XGBoost model")
plt.show()

#### Voting Classifier
The idea behind the VotingClassifier is to combine conceptually different machine learning classifiers and use a majority vote or the average predicted probabilities (soft vote) to predict the class labels. Such a classifier can be useful for a set of equally well performing model in order to balance out their individual weaknesses.

We create a voting classifier using three models: KNN, Random Forest, and XGBoost model. Results don't show any improvement over the three models. 

In [ ]:
from sklearn.ensemble import VotingClassifier

ensemble_knn_rf_xgb=VotingClassifier(estimators= [('KNN', knn), ('Random Forest', rf),('XGBoost',xgb)], voting='hard')
ensemble_knn_rf_xgb.fit(X_train,y_train)

#compute accuracy
print('The ensembled model with all the 3 classifiers is:',ensemble_knn_rf_xgb.score(X_test,y_test))

#make predictions
y_pred = ensemble_knn_rf_xgb.predict(X_test)
#Plot the confusion matrix
confusion_mat(y_pred, y_test)

## Stacking
Stacking is a way of combining multiple models, that introduces the concept of a meta learner. It is less widely used than bagging and boosting. Unlike bagging and boosting, stacking may be (and normally is) used to combine models of different types.

The point of stacking is to explore a space of different models for the same problem. The idea is that you can attack a learning problem with different types of models which are capable to learn some part of the problem, but not the whole space of the problem. So you can build multiple different learners and you use them to build an intermediate prediction, one prediction for each learned model. Then you add a new model which learns from the intermediate predictions the same target. This final model is said to be stacked on the top of the others, hence the name. Thus you might improve your overall performance, and often you end up with a model which is better than any individual intermediate model.

In [ ]:
from mlxtend.classifier import StackingCVClassifier

sclf = StackingCVClassifier(classifiers=[knn, rf, xgb, gbm], 
                          meta_classifier=rf)

print('10-fold cross validation:\n')

for clf, label in zip([knn, rf, xgb, gbm, rf], 
                      ['KNearest Neighbors',
                       'Random Forest', 
                        'XGB','GBM',
                       'MetaClassifier']):

    sclf_scores = model_selection.cross_val_score(clf, X, Y,
                                              cv=10, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (sclf_scores.mean(), sclf_scores.std(), label))

## Summarize results
We run the algorithms once again using StratifiedK-fold cross-validation and summarize our findings. 

In [ ]:
models = []
#models.append(('LR', LogisticRegression()))
models.append(('KNN', knn))
models.append(('DT', dt))
models.append(('RF', rf))
models.append(('GBM', gbm))
models.append(('XGB', xgb))
models.append(('Voting',ensemble_knn_rf_xgb))

 Every algorithm is tested and results are collected and printed. We then visualise the variation in the predictions of each algorithm using a boxplot.

In [ ]:
results = []
names = []

for name, model in models:
    kfold = model_selection.StratifiedKFold(n_splits=10, random_state=7)
    cv_results = model_selection.cross_val_score(model, X, Y, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    msg = "{}: {} ({})".format(name, cv_results.mean(), cv_results.std())
    print(msg)
    
#Add stacking results that we got previously
results.append(np.asarray(sclf_scores))
names.append('Stacking')

In [ ]:
# boxplot algorithm comparison
fig = plt.figure(figsize=(10,6))
fig.suptitle('Algorithm Comparison')
ax = sns.boxplot(x=names, y=results)
plt.xlabel('Classifiers')
plt.ylabel('Accuracy')
plt.show()

We obtained a best on average classification accuracy of approximately 77% using XgBoost and the Voting Classifier. We noted earlier from the confusion matrix of XGboost classifier that we there are 26 misclassified samples with 12 misses. One can further analyse those misclassified samples to better understand the model behaviorb. This is an extension that is not part of this notebook.

We end the analysis with classification report of our two best performing models.The classification report shows a representation of the main classification metrics on a per-class basis. This gives a deeper intuition of the classifier behavior over global accuracy which can mask functional weaknesses in one class of a multiclass problem. Visual classification reports are used to compare classification models to select models that are “redder”, e.g. have stronger classification metrics or that are more balanced.

Precision is the ability of a classiifer not to label an instance positive that is actually negative. For each class it is defined as as the ratio of true positives to the sum of true and false positives. 

Recall is the ability of a classifier to find all positive instances. For each class it is defined as the ratio of true positives to the sum of true positives and false negatives.

The F1 score is a weighted harmonic mean of precision and recall such that the best score is 1.0 and the worst is 0.0.

Based on the weighted F-1 score from the below two reports, XgBoost is a better classifier. So was the case when we considered number of False negatives samples. 

In [ ]:
visualizer = ClassificationReport(xgb,classes=['Not Diabetic','Diabetic'])

#visualizer.fit(X_train, y_train)  # Fit the visualizer and the model
visualizer.score(X_test, y_test)   # Evaluate the model on the test data
visualizer.poof()                  # Draw/show/poof the data

In [ ]:
visualizer = ClassificationReport(ensemble_knn_rf_xgb,classes=['Not Diabetic','Diabetic'])

visualizer.score(X_test, y_test)   # Evaluate the model on the test data
visualizer.poof()                  # Draw/show/poof the data